## 金融統計輯要- 信用卡篇

#### 目標網站:[信用卡業務資訊揭露](http://www.banking.gov.tw/ch/home.jsp?id=192&parentpath=0,4&mcustomize=multimessage_view.jsp&dataserno=21207&aplistdn=ou=disclosure,ou=multisite,ou=chinese,ou=ap_root,o=fsc,c=tw&dtable=Disclosure)

#### 說明: 
本練習為利用程式自動下載檔案，解壓縮後的excel檔經過簡單的ETL後存入資料庫。

#### 預期目標:

1. 找到目標網站,例如:105年9月下載連結[在此](http://www.fsc.gov.tw/fckdowndoc?file=/10509_信用卡重要資訊揭露.zip&flag=doc)
2. 解壓縮檔案(.zip)
3. ETL
4. 存進資料庫
5. 封裝以上流程

#### 下載檔案

連結指向.zip檔案，只要使用`request`就能取得內容。先讓檔案存在本地端，再做後續處理。


In [1]:
import requests # 
url_download = 'http://www.fsc.gov.tw/fckdowndoc?file=/10509_信用卡重要資訊揭露.zip&flag=doc'
res = requests.get(url_download)

In [2]:
zip_filename = './data/credit_card_' + str(10509) + '.zip' 

with open(zip_filename,'wb') as handle:
    if not res.ok:
        print 'OOPS, something wrong!'
    else:
        for block in res.iter_content(1024):
            handle.write(block)

#### 解壓縮

In [3]:
zip_dir = './data/credit_card_' + str(10509)

In [4]:
import zipfile

zip_ref = zipfile.ZipFile(zip_filename, 'r')
zip_ref.extractall(zip_dir)
zip_ref.close()

#### 處理下載的`.xlsx`檔案

In [5]:
import pandas as pd 

In [11]:
xlsx_path = zip_dir +'/'+ str(10509) + u'_信用卡重要資訊揭露.xlsx'
df = pd.read_excel(xlsx_path)
df

,信用卡重要業務及財務資訊(資訊揭露),Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"金額單位：新臺幣千元 ,卡",NaN,NaN,NaN,NaN,NaN,資料月份：105 年 9 月,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,金融機構名稱,流通卡數,有效卡數,當月發卡數,當月停卡數,循環信用,未到期,當月簽帳,當月預借,逾期三個月,逾期六個月,備抵呆帳,當月轉銷,當年度轉銷
4,NaN,,,,,餘額,分期付款,金額,現金金額,以上帳款占,以上帳款占,提足率,呆帳金額,呆帳金額
5,NaN,,,,,,餘額,,,應收帳款餘,應收帳款餘,(%),,累計至
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,額(含催收款),額(含催收款),NaN,NaN,資料月份
7,,,,,,,,,,之比率(%),之比率(%),,,NaN
8,臺灣銀行,229685,108569,1403,773,227510,8564,591923,1125,0.361745,0.136023,500.685,501,7515
9,臺灣土地銀行,226874,123797,3130,750,273104,49967,802498,842,0.349979,0.198352,1327.9,2632,13232


#### 清理資料
* 觀察需要擷取的資料位在第8-44行
* 重新命名欄位
* 定義欄位資料型態(dtype)

In [14]:
df_temp = df[8:44];
col_names = [u'金融機構名稱',u'流通卡數',u'有效卡數',u'當月發卡數',u'當月停卡數',u'循環信用餘額(仟)',
             u'未到期分期付款餘額(仟)',u'當月簽帳金額(仟)',u'當月預借現金金額(仟)',
             u'逾期三個月以上比率(%)',
             u'逾期六個月以上比率(%)',
             u'備抵呆帳提足率(%)',u'當月轉銷呆帳金額(仟)',u'當年度轉銷呆帳金額(仟)'
]
df_temp.columns = col_names

df_temp.head()

,金融機構名稱,流通卡數,有效卡數,當月發卡數,當月停卡數,循環信用餘額(仟),未到期分期付款餘額(仟),當月簽帳金額(仟),當月預借現金金額(仟),逾期三個月以上比率(%),逾期六個月以上比率(%),備抵呆帳提足率(%),當月轉銷呆帳金額(仟),當年度轉銷呆帳金額(仟)
8,臺灣銀行,229685,108569,1403,773,227510,8564,591923,1125,0.361745,0.136023,500.685,501,7515
9,臺灣土地銀行,226874,123797,3130,750,273104,49967,802498,842,0.349979,0.198352,1327.9,2632,13232
10,合作金庫商業銀行,434661,264265,4428,3048,632591,204449,2298644,3452,0.297598,0.216379,335.775,6157,34336
11,第一商業銀行,944592,632028,9874,8862,1422132,1010711,3768957,14321,0.169761,0,1874.76,5490,50783
12,華南商業銀行,860317,624604,11612,7488,746874,1722842,3471889,996,0.0669013,0,769.897,11522,36400


In [15]:
df_part1 = df_temp[[u'流通卡數',u'有效卡數',u'當月發卡數',u'當月停卡數',u'循環信用餘額(仟)',
         u'未到期分期付款餘額(仟)',u'當月簽帳金額(仟)',u'當月預借現金金額(仟)',u'當年度轉銷呆帳金額(仟)']].astype('int64')
df_part2 = df_temp[[ u'逾期三個月以上比率(%)']].astype('float64')
df_temp['yyyymm'] = '201609'
df_result = pd.concat([df_temp[['yyyymm']],df_temp[u'金融機構名稱'],df_part1,df_part2],axis=1)
df_result.head(10)

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,yyyymm,金融機構名稱,流通卡數,有效卡數,當月發卡數,當月停卡數,循環信用餘額(仟),未到期分期付款餘額(仟),當月簽帳金額(仟),當月預借現金金額(仟),當年度轉銷呆帳金額(仟),逾期三個月以上比率(%)
8,201609,臺灣銀行,229685,108569,1403,773,227510,8564,591923,1125,7515,0.361745
9,201609,臺灣土地銀行,226874,123797,3130,750,273104,49967,802498,842,13232,0.349979
10,201609,合作金庫商業銀行,434661,264265,4428,3048,632591,204449,2298644,3452,34336,0.297598
11,201609,第一商業銀行,944592,632028,9874,8862,1422132,1010711,3768957,14321,50783,0.169761
12,201609,華南商業銀行,860317,624604,11612,7488,746874,1722842,3471889,996,36400,0.066901
13,201609,彰化商業銀行,433112,210300,2596,3545,368688,163663,1202909,490,20324,0.345814
14,201609,上海商業儲蓄銀行,355758,177470,1428,1781,736921,183239,1046060,7095,21170,0.872190
15,201609,台北富邦商業銀行,2381409,1699013,105020,26099,6311070,8321286,15414178,44722,212163,0.163105
16,201609,國泰世華商業銀行,5819486,3987545,167547,30382,14403009,10716675,28436766,226868,274472,0.128559
17,201609,高雄銀行,9995,4621,119,40,7595,112,161677,181,915,0.219022


* 給予欄位正確SQL資料型態

In [16]:
from sqlalchemy.dialects.mssql import VARCHAR,NVARCHAR,FLOAT,INTEGER 

In [17]:
dict_dtype={
        u'yyyymm':NVARCHAR(length=6),
        u'金融機構名稱':NVARCHAR(length=20),
        u'流通卡數':INTEGER,
        u'有效卡數':INTEGER,
        u'當月發卡數':INTEGER,
        u'當月停卡數':INTEGER,
        u'循環信用餘額(仟)':INTEGER,
        u'未到期分期付款餘額(仟)':INTEGER,
        u'當月簽帳金額(仟)':INTEGER,
        u'當月預借現金金額(仟)':INTEGER,
        u'逾期三個月以上比率(%)':FLOAT,
        u'當月轉銷呆帳金額(仟)':INTEGER
}

#### 連結MS資料庫
* 寫入資料

In [18]:
import pyodbc 
import sqlalchemy 
connect_w = lambda x:pyodbc.connect('DRIVER={SQL Server};SERVER=dbm_public;DATABASE=External;PWD=01060728;CHARSET=utf8;',
                                     unicode_result=True)
conn_w = sqlalchemy.create_engine('mssql://',creator=connect_w).connect()

In [20]:
df_result.to_sql(u'測試table',conn_w,index = False, if_exists = 'replace',dtype=dict_dtype); # 若要資料要不斷累加，if_exists = 'append'

## 包裝成函數

In [43]:
import requests
import zipfile
import pandas
import pyodbc,sqlalchemy
from sqlalchemy.dialects.mssql import VARCHAR,NVARCHAR,FLOAT,INTEGER 



def getBankCCinfo(yyymm):
    ''' 下載並解壓縮，信用卡重要業務及財務資訊揭露.zip
    參數:
        民國年月- 10509    
        
    '''
    assert isinstance(yyymm,int),'PLEASE Enter INT tyep such as 10508' # 確認輸入為整數
    url_download = 'http://www.fsc.gov.tw/fckdowndoc?file=/{}_信用卡重要資訊揭露.zip&flag=doc'.format(yyymm)
    res = requests.get(url_download)
    
    zip_dir = './data/credit_card_' + str(yyymm)
    zip_filename = './data/credit_card_' + str(yyymm) + '.zip' 
    
    ### 存檔
    with open(zip_filename,'wb') as handle:
        if not res.ok:
            print 'OOPS, something wrong!'
        else:
            for block in res.iter_content(1024):
                handle.write(block)
    
    ### 解壓縮    
    zip_ref = zipfile.ZipFile(zip_filename, 'r')
    zip_ref.extractall(zip_dir)
    zip_ref.close()

    
def parseXLSXtoSQL(yyymm):
    
    assert isinstance(yyymm,int) #確認輸入為整數
    
    zip_dir = './data/credit_card_' + str(yyymm)
    xlsx_path = zip_dir +'/'+ str(yyymm) + u'_信用卡重要資訊揭露.xlsx'
    df = pd.read_excel(xlsx_path)
    df_temp = df[8:44];
    col_names = [u'金融機構名稱',u'流通卡數',u'有效卡數',u'當月發卡數',u'當月停卡數',u'循環信用餘額(仟)',
                 u'未到期分期付款餘額(仟)',u'當月簽帳金額(仟)',u'當月預借現金金額(仟)',
                 u'逾期三個月以上比率(%)',
                 u'逾期六個月以上比率(%)',
                 u'備抵呆帳提足率(%)',u'當月轉銷呆帳金額(仟)',u'當年度轉銷呆帳金額(仟)'
    ]
    df_temp.columns = col_names
    
    df_part1 = df_temp[[u'流通卡數',u'有效卡數',u'當月發卡數',u'當月停卡數',u'循環信用餘額(仟)',
         u'未到期分期付款餘額(仟)',u'當月簽帳金額(仟)',u'當月預借現金金額(仟)',u'當年度轉銷呆帳金額(仟)']].astype('int64')
    df_part2 = df_temp[[ u'逾期三個月以上比率(%)']].astype('float64')
    
    
    yyyy = 1911 + int(str(yyymm)[:3]) # 105 --> 2016
    yyyymm = str(yyyy) + str(yyymm)[-2:]
    
    
    df_temp['yyyymm'] = yyyymm
    df_result = pd.concat([df_temp[['yyyymm']],df_temp[u'金融機構名稱'],df_part1,df_part2],axis=1)
    dict_dtype={
        u'yyyymm':NVARCHAR(length=6),
        u'金融機構名稱':NVARCHAR(length=20),
        u'流通卡數':INTEGER,
        u'有效卡數':INTEGER,
        u'當月發卡數':INTEGER,
        u'當月停卡數':INTEGER,
        u'循環信用餘額(仟)':INTEGER,
        u'未到期分期付款餘額(仟)':INTEGER,
        u'當月簽帳金額(仟)':INTEGER,
        u'當月預借現金金額(仟)':INTEGER,
        u'逾期三個月以上比率(%)':FLOAT,
        u'當月轉銷呆帳金額(仟)':INTEGER
    }
    
    df_result.to_sql(u'測試table',conn_w,index = False, if_exists = 'replace',dtype=dict_dtype)
    return df_result


if __name__ == '__main__':

    connect_w = lambda x:pyodbc.connect('DRIVER={SQL Server};SERVER=dbm_public;DATABASE=External;PWD=01060728;CHARSET=utf8;',
                                     unicode_result=True)
    conn_w = sqlalchemy.create_engine('mssql://',creator=connect_w).connect()
    
    getBankCCinfo(10508)
    parseXLSXtoSQL(10508)
    
    conn_w.close()

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda2\lib\site-packages\sqlalchemy\sql\sqltypes.py:185: SAWarning: Unicode type received non-unicode bind param value '201608'. (this warning may be suppressed after 10 occurrences)
  (util.ellipses_string(value),))
